## Interpolate input datasets to radar data locations

The purpose of this notebook is to load external input datasets and interpolate them onto the locations of the provided radar RSSNR data.

This notebook depends on external data sources. There is a script in `external_datasets/download_datasets.sh` that automates the process of downloading the required datasets.

This notebook is intended to be run once per dataset (i.e. per combination of radar data source and ice sheet).

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dask
from dask.distributed import Client, LocalCluster

client = Client(n_workers=1, threads_per_worker=4, memory_limit='60GB') # Note that `memory_limit` is the limit **per worker**.

client # If you click the dashboard link in the output, you can monitor real-time progress and get other cool visualizations.

In [25]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial
import cartopy
import cartopy.crs as ccrs

from interpolation_utils import interpolate_nearest_from_grid

In [26]:
# Map projections for plotting
crs_3031 = cartopy.crs.Stereographic(central_latitude=-90, true_scale_latitude=-71) # All Antarctic data will be projected (if needed) to this
crs_3413 = ccrs.Stereographic(central_latitude=90, central_longitude=-45, true_scale_latitude=70) # All Greenland data will be projected (if needed) to this
crs_lonlat = cartopy.crs.PlateCarree()

In [27]:
# Dataset options

# CReSIS / Antarctica
source_csv_path, decimate_by_n, output_nc_path, dataset = 'data_preprocessing/snr_data_cresis_ais.csv', 5, 'data_preprocessing/snr_data_cresis_ais_with_inputs.nc', 'antarctica'

# CReSIS / Greenland
#source_csv_path, decimate_by_n, output_nc_path, dataset = 'data_preprocessing/snr_data_cresis_gis.csv', 5, 'data_preprocessing/snr_data_cresis_gis_with_inputs.nc', 'greenland'

# UTIG / Antarctica
#source_csv_path, decimate_by_n, output_nc_path, dataset = 'external_datasets/utig_rssnr/snr.csv', 10, 'data_preprocessing/snr_data_utig_ais_with_inputs.nc', 'antarctica'

In [ ]:
# Load required surface SNR data

df_tmp = pd.read_csv(source_csv_path)[::decimate_by_n]
ds_radar = xr.Dataset.from_dataframe(df_tmp)
ds_radar = ds_radar.rename({'thickness': 'picked_thickness', 'surface': 'picked_surface', 'bottom': 'picked_bottom'})
ds_radar

In [ ]:
median_spacing = np.sqrt(np.median(np.diff(ds_radar.x)**2 + np.diff(ds_radar.y)**2))
print(f"Median along-track radar spacing after decimation by {decimate_by_n}: {median_spacing:.2f} m")

In [ ]:
# print 10th and 90th percentiles of ds_radar x and y
print("10th percentile of ds_radar x:", np.percentile(ds_radar.x, 10))
print("90th percentile of ds_radar x:", np.percentile(ds_radar.x, 90))
print("10th percentile of ds_radar y:", np.percentile(ds_radar.y, 10))
print("90th percentile of ds_radar y:", np.percentile(ds_radar.y, 90))

In [31]:
if dataset == 'antarctica':
    # Load BedMachine datasets
    ds_bm = xr.open_dataset("external_datasets/BedMachineAntarctica-v3.nc")

    # Load Rignot surface velocity datasets
    ds_vel = xr.open_dataset("external_datasets/antarctic_ice_vel_phase_map_v01.nc")
elif dataset == 'greenland':
    # Load BedMachine datasets
    ds_bm = xr.open_dataset("external_datasets/BedMachineGreenland-v5.nc")

    # Load ITS_LIVE surface velocity
    ds_vel = xr.open_dataset("external_datasets/ITS_LIVE_velocity_120m_RGI05A_0000_v02.nc")

In [32]:
# # Load ERA5 t2m data
ds_era5 = xr.open_dataset("external_datasets/era5_t2m_ensemble.nc")

t2m_mean = ds_era5.t2m.mean(dim='valid_time').mean(dim='number')
t2m_std = ds_era5.t2m.mean(dim='valid_time').std(dim='number')

# Combined t2m_mean and t2m_std into a single dataset
ds_t2m = xr.Dataset({'t2m_mean': t2m_mean, 't2m_std': t2m_std})

In [33]:
if dataset == 'antarctica':
    # Calculate magnitude (speed) and error per NSIDC-0754 user guide:
    # https://nsidc.org/sites/default/files/nsidc-0754-v001-userguide.pdf
    ds_vel['speed'] = np.sqrt(ds_vel['VX']**2 + ds_vel['VY']**2)
    ds_vel['speed_err'] = np.sqrt(ds_vel['ERRX']**2 + ds_vel['ERRY']**2)
elif dataset == 'greenland':
    # ITS_LIVE velocity data already has magnitude and magnitude error calculated, just need to re-name
    ds_vel['speed'] = ds_vel['v']
    ds_vel['speed_err'] = ds_vel['v_error']

In [34]:
# NN interpolate from BedMachine data
ds_radar['thickness'], ds_radar['thickness_err'], ds_radar['bm_mask'], ds_radar['surface'] = interpolate_nearest_from_grid(ds_bm, ds_radar,
                                                                                                        ['thickness', 'errbed', 'mask', 'surface'])

In [35]:
# NN interpolate from surface velocity data

ds_radar['speed'], ds_radar['speed_err'] = interpolate_nearest_from_grid(ds_vel, ds_radar, ['speed', 'speed_err'])

In [36]:
if dataset == 'antarctica':
    target_crs = crs_3031
    crs_name = 'EPSG:3031'
elif dataset == 'greenland':
    target_crs = crs_3413
    crs_name = 'EPSG:3413'

ds_radar['t2m'], ds_radar['t2m_err'] = interpolate_nearest_from_grid(ds_t2m, ds_radar, ['t2m_mean', 't2m_std'],
                                            source_crs=crs_lonlat, target_crs=target_crs, y_name='latitude', x_name='longitude')

In [ ]:
# Sanity check: Plot interpolated thickness points over BedMachine thickness map

fig, ax = plt.subplots(figsize=(6,6), subplot_kw=dict(projection=target_crs))

vmin = np.nanmin(ds_bm['thickness'])
vmax = np.nanmax(ds_bm['thickness'])

ds_bm['surface'][::10, ::10].plot.pcolormesh(ax=ax, alpha=0.8, vmin=vmin, vmax=vmax, cmap='turbo')
ax.coastlines(resolution='10m', color='white', linewidth=0.5)

ax.scatter(x=ds_radar.x, y=ds_radar.y, c=ds_radar.surface, vmin=vmin, vmax=vmax, s=1, cmap='turbo')

# Axes and coastlines
ax.set_xlabel(f'{crs_name} X [km]')
ax.set_ylabel(f'{crs_name} Y [km]')

# if dataset == 'greenland':
#     ax.set_xlim(-6e5, 6e5)
#     ax.set_ylim(-3.5e6, -0.5e6)

ax.set_xticks(ax.get_xticks(), crs=target_crs)
ax.set_yticks(ax.get_yticks(), crs=target_crs)
ax.tick_params(axis='x', rotation=90)
# Show axis ticks in kilometers
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/1000:.0f}'))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/1000:.0f}'))


In [ ]:
# Sanity check: Plot interpolated surface velocity points over MEaSUREs surface velocity map
fig, ax = plt.subplots(figsize=(6,6), subplot_kw=dict(projection=target_crs))
vmin = 0 #np.nanmin(np.log(ds_vel['speed']))
vmax = np.nanmax(np.log(ds_vel['speed']))

cmap='turbo'

np.log(ds_vel['speed'][::10, ::10]).plot.pcolormesh(ax=ax, alpha=0.8, vmin=vmin, vmax=vmax, cmap=cmap)
ax.coastlines(resolution='10m', color='white', linewidth=0.5)

ax.scatter(x=ds_radar.x, y=ds_radar.y, c=np.log(ds_radar.speed), vmin=vmin, vmax=vmax, s=1, cmap=cmap)

# Axes
ax.set_xlabel(f'{crs_name} X [km]')
ax.set_ylabel(f'{crs_name} Y [km]')
ax.set_xticks(ax.get_xticks(), crs=target_crs)
ax.set_yticks(ax.get_yticks(), crs=target_crs)
ax.tick_params(axis='x', rotation=90)
# Show axis ticks in kilometers
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/1000:.0f}'))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/1000:.0f}'))


In [ ]:
ds_radar['t2m'].min(), ds_radar['t2m'].max()

In [ ]:
# Sanity check: Plot interpolated t2m points over ERA5 t2m map

fig, ax = plt.subplots(figsize=(6,6), subplot_kw=dict(projection=target_crs))

vmin = np.nanmin(ds_radar['t2m'])
vmax = np.nanmax(ds_radar['t2m'])
cmap='turbo'

pcm = ax.pcolormesh(ds_t2m['longitude'], ds_t2m['latitude'], ds_t2m['t2m_mean'], vmin=vmin, vmax=vmax, cmap=cmap, alpha=0.4, transform=crs_lonlat)
ax.coastlines(resolution='10m', color='white', linewidth=0.5)
sc = ax.scatter(x=ds_radar.x, y=ds_radar.y, c=ds_radar.t2m, vmin=vmin, vmax=vmax, s=1, cmap=cmap)
fig.colorbar(pcm, ax=ax, label='Temperature at 2 meters [K]')

if dataset == 'antarctica':
    ax.set_xlim(-3000e3, 3000e3)
    ax.set_ylim(-3000e3, 3000e3)
elif dataset == 'greenland':
    ax.set_xlim(-6e5, 6e5)
    ax.set_ylim(-3.5e6, -0.5e6)

# Axes
ax.set_xlabel(f'{crs_name} X [km]')
ax.set_ylabel(f'{crs_name} Y [km]')
ax.set_xticks(ax.get_xticks(), crs=target_crs)
ax.set_yticks(ax.get_yticks(), crs=target_crs)
ax.tick_params(axis='x', rotation=90)
# Show axis ticks in kilometers
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/1000:.0f}'))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x/1000:.0f}'))

In [41]:
# fig, ax = plt.subplots(figsize=(8,8))

# ax.scatter(np.log(ds_radar.speed), ds_radar.snr, s=1, alpha=0.01)
# ax.set_xlabel('Surface Velocity [log(m/yr)]')
# ax.set_ylabel('RSSNR [dB]')
# ax.set_title(f"Mean Radius: {mean_radius/1e3} km")

In [42]:
# fig, ax = plt.subplots()

# ds_bm_ais.errbed.plot.hist(bins=30, density=True, ax=ax, label='Antarctica')
# ds_bm_gis.errbed.plot.hist(bins=30, density=True, ax=ax, label='Greenland', alpha=0.5)

# print(np.min(ds_bm_ais.errbed))
# print(np.min(ds_bm_gis.errbed))

# ax.set_title('BedMachine error histogram')
# ax.legend()

In [43]:
ds_radar.to_netcdf(output_nc_path)

In [ ]:
output_nc_path